# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [60]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Markers, please use your onw API keys to run the script, thank you!
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [61]:
# Load the csv exported in Part I to a DataFrame
csv_path = "../output_files/cities_weather.csv"
cities_weather_df = pd.read_csv(csv_path)

cities_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Praia,14.9215,-23.5087,79.39,78,100,4.36,CV,1631404873
1,Betsiamites,48.9413,-68.6460,58.50,82,15,9.98,CA,1631405405
2,Bernalillo,35.0500,-106.6673,96.01,13,20,10.36,US,1631405407
3,Albany,42.6001,-73.9662,65.91,76,17,4.56,US,1631405135
4,El Puerto de Santa María,36.5939,-6.2330,72.25,88,0,0.00,ES,1631405116
...,...,...,...,...,...,...,...,...,...
533,Port,-7.0349,29.7638,75.00,57,54,1.90,CD,1631405887
534,Aksarka,66.5606,67.7975,38.17,83,100,9.75,RU,1631405888
535,Lakes,61.6071,-149.3086,61.39,67,75,1.01,US,1631405889
536,Rocha,-34.4833,-54.3333,52.30,97,69,7.25,UY,1631405891


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [62]:
# Configure the gmaps
gmaps.configure(api_key = g_key)

# Create variable to store Latitude and Longitude
city_lat_lng = cities_weather_df[["Lat", "Lng"]]
humidity = cities_weather_df["Humidity"].astype(float)

# Create figure layout options in dictionary
figure_layout = {"width": "100%",
                 "height": "600px",
                 "padding": "1px"}

# Create heat layer
fig = gmaps.figure(layout = figure_layout, center = (20.0, 10.0), zoom_level = 2)
heat_layer = gmaps.heatmap_layer(city_lat_lng,
                                 weights = humidity,
                                 max_intensity = np.max(humidity),
                                 dissipating = False,
                                 point_radius = 2)

# add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [63]:
# Create dataframe for ideal locations with cities meets the requirements:
# A max temperature between 70(F) - 80(F)
# Wind speed less than 10 mph
# Zero cloudiness
vacation_cities_df = cities_weather_df.loc[(cities_weather_df["Max Temp"] >= 70) &
                                           (cities_weather_df["Max Temp"] <= 80) &
                                           (cities_weather_df["Wind Speed"] <= 10) &
                                           (cities_weather_df["Cloudiness"] == 0)].dropna()

vacation_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,El Puerto de Santa María,36.5939,-6.2330,72.25,88,0,0.00,ES,1631405116
22,Flinders,-34.5833,150.8552,77.20,39,0,7.00,AU,1631405422
26,Seoul,37.5683,126.9778,72.88,70,0,1.30,KR,1631405294
101,Yanta,36.2411,115.6672,76.41,85,0,5.93,CN,1631405490
128,Belmonte,-15.8631,-38.8828,73.78,81,0,8.70,BR,1631405512
166,Ballina,-28.8667,153.5667,77.77,56,0,9.24,AU,1631405544
216,El Puerto de Santa María,36.5939,-6.2330,71.26,88,0,0.00,ES,1631405586
260,Tabas,33.5959,56.9244,77.29,23,0,5.17,IR,1631405626
273,Kaifeng,34.7911,114.3483,74.71,81,0,5.10,CN,1631405638
369,Antiguo,43.3134,-2.0018,71.04,84,0,1.99,ES,1631405717


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:
# Drop un-needed columns from the original data frame
hotel_df = vacation_cities_df.loc[:, ["City", "Lat", "Lng", "Country"]]
hotel_df["Hotel Name"] = ""

# Base url for Google Places API - Nearby Search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Set parameters for the API call
params = {"keyword": "hotel",
          "type": "lodging",
          "key": g_key,
          "radius": 5000,
          "location": ""}

# Perform API calls to collect hotel name
for index, row in hotel_df.iterrows():
    
    # Insert city coordinates into parameters
    params["location"] = str(row[1]) +","+ str(row[2])
    
    # API call
    response = requests.get(base_url, params).json()
    
    # Store into results part of dictionary
    hotel_results = response["results"]
    
    # Print hotel name and add to data frame if hotel is successfully found
    try:
        print(f"{row['City']}, {row['Country'].upper()} hotel: {hotel_results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = hotel_results[0]["name"]
    
    # Print error statement if hotels cannot be found
    except (KeyError, IndexError):
        print(f"No hotel found for this city within 5000 meters.")
    
hotel_df

El Puerto de Santa María, ES hotel: Soho Boutique Vistahermosa
Flinders, AU hotel: Lakeview Hotel Motel
Seoul, KR hotel: Hotel Skypark Kingstown Dongdaemun
Yanta, CN hotel: Hanlin Boutique Hotel
Belmonte, BR hotel: Pousada Rio
Ballina, AU hotel: Ramada Hotel & Suites by Wyndham Ballina Byron
El Puerto de Santa María, ES hotel: Soho Boutique Vistahermosa
Tabas, IR hotel: Narenjestan Hotel
Kaifeng, CN hotel: New Century Grand Hotel Kaifeng （North Gate）
Antiguo, ES hotel: Zenit San Sebastián
Pakpattan, PK hotel: Mehran Hotel
Marrakesh, MA hotel: Sofitel Marrakech Lounge & Spa
No hotel found for this city within 5000 meters.
Narrabri, AU hotel: Crossroads Hotel
Upington, ZA hotel: Protea Hotel by Marriott Upington
No hotel found for this city within 5000 meters.


,City,Lat,Lng,Country,Hotel Name
4,El Puerto de Santa María,36.5939,-6.2330,ES,Soho Boutique Vistahermosa
22,Flinders,-34.5833,150.8552,AU,Lakeview Hotel Motel
26,Seoul,37.5683,126.9778,KR,Hotel Skypark Kingstown Dongdaemun
101,Yanta,36.2411,115.6672,CN,Hanlin Boutique Hotel
128,Belmonte,-15.8631,-38.8828,BR,Pousada Rio
166,Ballina,-28.8667,153.5667,AU,Ramada Hotel & Suites by Wyndham Ballina Byron
216,El Puerto de Santa María,36.5939,-6.2330,ES,Soho Boutique Vistahermosa
260,Tabas,33.5959,56.9244,IR,Narenjestan Hotel
273,Kaifeng,34.7911,114.3483,CN,New Century Grand Hotel Kaifeng （North Gate）
369,Antiguo,43.3134,-2.0018,ES,Zenit San Sebastián


In [65]:
cities_count = hotel_df["City"].nunique()
print(f"%s cities match ideal vaction city requirements." % cities_count)

15 cities match ideal vaction city requirements.


In [66]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [67]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', padding='1px', width='100%'))